### Using just SQL

Having a quality data in the database is sufficient for any investigation. We can use any programming language we want. 
For example, it is possible to replicate Collective2 Scoring Workbench functionality using just SQL.
See the following examples.

#### SQL - Example 1

This is an SQL code for Daniil's formula above. Using some MySql UI, we can inspect all available data of the each selected strategy. 
It's an ideal form for exploring the data in details.
We can sorting any column up and down when we are trying to find extremes. We can modify formula or selection easily. We can filter results in IDE...

If we are working with a local database, we can see results immediately. 

~~~sql
#  ---------- Plain with all data  ----------
SELECT 
    StrategyId, 
    StrategyName, 
    Added,
    -- Risk
    (20 - MaxDrawdownPcnt * (20/45)) 
    + if(isnull(MaxWorstLossPercentEquity060), 0, (20 - MaxWorstLossPercentEquity060 * (20.0/8)))
    + (10 - OptionPercent * (10/1))
    + if(ShortOptionsCovered >= 100,10,ShortOptionsCovered / 100 * 10)
    -- Behavior
    + if(AgeDays >= 720, 40,AgeDays/720 * 40)
    + (20 - DailyMaxLevMax * (20/10)) 
    + if (TradeDaysAgo <= 60, 20, 0)
    + if (PcntMonthsProfitable >= 100 , 5 , PcntMonthsProfitable / 100 * 5)
    + if (NumTrades >= 100 , 20 , NumTrades / 100 * 20)
    + if ((DeltaEquity90Days <= 0 and DeltaEquity90Days > -7) , 5 , 0) 
	-- Profitability 
    + if(AnnReturn > 0.6 , 20 , AnnReturn / 0.6 * 20 ) as Score, 
    c2score_ScoringWorkbenchView.*
FROM c2score_ScoringWorkbenchView
where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
having Score > 0
order by Score desc;
~~~

![Image](./MySql_Daniil01_plain.png)

---
#### SQL - Example 2



If the used IDE can export data in the HTML format, we can include Collective2 pictures too:

~~~mysql
# ------------- Simple  ----------
SELECT 
    concat("<p>Chart as of ",CURRENT_DATE(),"</p><a href='https://collective2.com/details/",StrategyId,"' target='_blank'>",
        "<img src='https://collective2.com/cgi-perl/xcharts200.mpl?want=nft&width=200&height=150&systemid=",StrategyId,"'/></a>") as Chart,
    StrategyId, 
    StrategyName, 
    Added,
    -- Risk
    (20 - MaxDrawdownPcnt * (20/45)) 
    + if(isnull(MaxWorstLossPercentEquity060), 0, (20 - MaxWorstLossPercentEquity060 * (20.0/8)))
    + (10 - OptionPercent * (10/1))
    + if(ShortOptionsCovered >= 100,10,ShortOptionsCovered / 100 * 10)
    -- Behavior
    + if(AgeDays >= 720, 40,AgeDays/720 * 40)
    + (20 - DailyMaxLevMax * (20/10)) 
    + if (TradeDaysAgo <= 60, 20, 0)
    + if (PcntMonthsProfitable >= 100 , 5 , PcntMonthsProfitable / 100 * 5)
    + if (NumTrades >= 100 , 20 , NumTrades / 100 * 20)
    + if ((DeltaEquity90Days <= 0 and DeltaEquity90Days > -7) , 5 , 0) 
	-- Profitability 
    + if(AnnReturn > 0.6 , 20 , AnnReturn / 0.6 * 20 ) as Score
    -- , c2score_ScoringWorkbenchView.*
FROM c2score_ScoringWorkbenchView
where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
having Score > 0
order by Score desc
limit 30;
~~~


Export data to the HTML file using MySQL Workbench:

![Image](./MySql_Daniil01_Simple.png)

#### [You can see a generated page here.](https://svancara.github.io/C2QuantAPI/daniilFormulaInMySql_Simple.html)

---
#### SQL - Example 3

Using a little more sophisticated SQL command, we can generate also intermediate score like Daniil's Risk, Behavior and Profitability.

~~~mysql
# ------------- Structured ----------
SELECT 
    concat("<p>Chart as of ",CURRENT_DATE(),"</p><a href='https://collective2.com/details/",StrategyId,"' target='_blank'>",
        "<img src='https://collective2.com/cgi-perl/xcharts200.mpl?want=nft&width=200&height=150&systemid=",StrategyId,"'/></a>") as Chart,
    StrategyId, 
    StrategyName, 
    Added,
    @Risk := (20 - MaxDrawdownPcnt * (20/45)) 
    + if(isnull(MaxWorstLossPercentEquity060), 0, (20 - MaxWorstLossPercentEquity060 * (20.0/8)))
    + (10 - OptionPercent * (10/1))
    + if(ShortOptionsCovered >= 100,10,ShortOptionsCovered / 100 * 10) 
    as Risk,
    @Behavior := if(AgeDays >= 720, 40,AgeDays/720 * 40)
    + (20 - DailyMaxLevMax * (20/10)) 
    + if (TradeDaysAgo <= 60, 20, 0)
    + if (PcntMonthsProfitable >= 100 , 5 , PcntMonthsProfitable / 100 * 5)
    + if (NumTrades >= 100 , 20 , NumTrades / 100 * 20)
    + if ((DeltaEquity90Days <= 0 and DeltaEquity90Days > -7) , 5 , 0) 
    as Behavior,
    @Profitability :=  if(AnnReturn > 0.6 , 20 , AnnReturn / 0.6 * 20 )
      as Profitability,    
    @Risk + @Behavior + @Profitability as Score
    -- , c2score_ScoringWorkbenchView.*
FROM c2score_ScoringWorkbenchView
where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
having Score > 0
order by Score desc
limit 30;
~~~

A result can be investigated in the IDE or exported to the HTML format showing Collective2 charts.

#### [You can see a generated page here.](https://svancara.github.io/C2QuantAPI/daniilFormulaInMySql_Structured.html)


---
#### SQL - Example 4

We can generate many different variants of HTML pages using just the SQL language.

~~~mysql
# ------------- Detailed ----------
SELECT 
    concat("<p>Chart as of ",CURRENT_DATE(),"</p><a href='https://collective2.com/details/",StrategyId,"' target='_blank'>",
        "<img src='https://collective2.com/cgi-perl/xcharts200.mpl?want=nft&width=200&height=150&systemid=",StrategyId,"'/></a>") as Chart,
    StrategyId, 
    StrategyName, 
    Added,
    concat("<pre>",
     "AgeDays = ",AgeDays,"<br>",
     "AnnReturn = ",AnnReturn,"<br>",
     "DailyMaxLevMax = ",DailyMaxLevMax,"<br>",
     "DeltaEquity90Days = ",DeltaEquity90Days,"<br>",
     "MaxDrawdownPcnt = ",MaxDrawdownPcnt,"<br>",
     "MaxWorstLossPercentEquity060 = ",MaxWorstLossPercentEquity060,"<br>",
     "NumTrades = ",NumTrades,"<br>",
     "OptionPercent = ",OptionPercent,"<br>",
     "PcntMonthsProfitable = ",PcntMonthsProfitable,"<br>",
     "ShortOptionsCovered = ",ShortOptionsCovered,"<br>",
     "TradeDaysAgo = ",TradeDaysAgo,"<br>",
     "</pre>"
     ) as Details,
    @Risk := (20 - MaxDrawdownPcnt * (20/45)) 
    + if(isnull(MaxWorstLossPercentEquity060), 0, (20 - MaxWorstLossPercentEquity060 * (20.0/8)))
    + (10 - OptionPercent * (10/1))
    + if(ShortOptionsCovered >= 100,10,ShortOptionsCovered / 100 * 10)
    as Risk,
    @Behavior := if(AgeDays >= 720, 40,AgeDays/720 * 40)
    + (20 - DailyMaxLevMax * (20/10)) 
    + if (TradeDaysAgo <= 60, 20, 0)
    + if (PcntMonthsProfitable >= 100 , 5 , PcntMonthsProfitable / 100 * 5)
    + if (NumTrades >= 100 , 20 , NumTrades / 100 * 20)
    + if ((DeltaEquity90Days <= 0 and DeltaEquity90Days > -7) , 5 , 0) 
    as Behavior,
    @Profitability :=  if(AnnReturn > 0.6 , 20 , AnnReturn / 0.6 * 20 )
      as Profitability,    
    @Risk + @Behavior + @Profitability as Score
FROM c2score_ScoringWorkbenchView
where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
having Score > 0
order by Score desc
limit 30;
~~~

#### [You can see a generated page here.](https://svancara.github.io/C2QuantAPI/daniilFormulaInMySql_Details.html)
